# Week3_2 Assignment

## [BASIC](#Basic) 
- 한국어 코퍼스를 로드해 **WordPiece Tokenzier를 학습**시킬 수 있다.
- 학습된 모델을 로드해 **encoding과 decoding을 수행**할 수 있다. 



### Reference
- [BertWordPieceTokenizer 학습 소개 한국어 블로그](https://monologg.kr/2020/04/27/wordpiece-vocab/)
- [huggingface python train tutorial](https://github.com/huggingface/tokenizers/blob/master/bindings/python/examples/train_bert_wordpiece.py)

In [1]:
import os 
import sys
import pandas as pd
import numpy as np

In [2]:
!pip install tokenizers

     |████████████████████████████████| 6.5 MB 9.4 MB/s 


In [3]:
import torch
from tokenizers import BertWordPieceTokenizer

In [4]:
# seed
seed = 7777
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [5]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla K80
cuda


## Basic

### 데이터 다운로드
- 내 구글 드라이브에 데이터를 다운 받은 후 코랩에 드라이브를 마운트하면 데이터를 영구적으로 사용할 수 있다. 
- [데이터 다운로드 출처](https://ratsgo.github.io/embedding/downloaddata.html)

In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
cd "/content/drive/MyDrive/pre_onboarding"

/content/drive/MyDrive/pre_onboarding


In [ ]:
# 한국어 위키피디아 데이터 (토크나이즈되지 않은 텍스트) 로드
!pip install gdown

In [8]:
# !gdown https://drive.google.com/u/0/uc?id=1kUecR7xO7bsHFmUI6AExtY5u2XXlObOG # acces denied : 용량 너무 커서 그런 듯. 링크 들어가서 직접 다운받고 구글 드라이브 폴더(pre_onboarding)에 옮겨놓음
!unzip processed.zip

Archive:  processed.zip
replace processed/space-correct.model? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: processed/space-correct.model  
replace processed/processed_korquad.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: processed/processed_korquad.txt  
replace processed/processed_ratings_train.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: processed/processed_ratings_train.txt  
replace processed/processed_ratings_test.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: processed/processed_ratings_test.txt  
  inflating: processed/processed_wiki_ko.txt  y

replace processed/processed_ratings.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: processed/processed_ratings.txt  
replace processed/corrected_ratings_corpus.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: processed/corrected_ratings_corpus.txt  
replace processed/soyword.model? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: processed/soyword.model  
replace processed

In [9]:
_CUR_DIR = os.path.abspath(os.curdir)
print(f"My current directory : {_CUR_DIR}")
_DATA_DIR = os.path.join(_CUR_DIR, "processed")

My current directory : /content/drive/MyDrive/pre_onboarding


### 한국어 위키피디아 코퍼스로 WordPiece tokenizer 학습
- 한국어 위키 

In [10]:
# processed_wiki_ko.txt 파일 불러오기

docs = open('processed/processed_wiki_ko.txt', 'r')
docs = docs.readlines()

In [11]:
print(f"# wiki documents: {len(docs):,}")

# wiki documents: 311,237


In [12]:
# Word Piece Tokenizer 인스턴스 생성
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False, # 악센트가 있는 character의 악센트를 제거하려면? (ex. é → e) (strip_accent와 lowercase는 함께 False 지정해줘야 한다.)
    lowercase=False, # 한국어는 대소문자가 없는데 소문자 변환이 필요한지?
)

In [13]:
# train
# files: 'processed_wiki_ko.txt'
# vocab_size: 30,000
# min_frequency: 2
# special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
# limit_alphabet: 1,000
# wordpieces_prefix: '##'

tokenizer.train(
    files = 'processed/processed_wiki_ko.txt',
    vocab_size = 30000,
    min_frequency = 2,
    show_progress = True,
    special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    limit_alphabet = 1000,
    wordpieces_prefix = '##'
)

In [14]:
tokenizer.save_model(".", "wordpiece")

['./wordpiece-vocab.txt']

### Encoding
- 저장된 토크나이즈 파일을 로드해 `BertWordPieceTokenizer` 인스턴스를 생성하고 다음을 수행하자. 
    - 사전(vocab)의 단어 개수를 출력
    - 문장을 토크나이징한 후 토큰 id와 토큰 string을 출력

In [15]:
tokenizer = BertWordPieceTokenizer(
    vocab = "/content/drive/MyDrive/pre_onboarding/wordpiece-vocab.txt",
    lowercase = False,
    strip_accents = False,
)

In [16]:
# 사전 단어 개수 출력
tokenizer.get_vocab_size()

30000

In [17]:
tokenizer.get_vocab()

{'의회는': 17340,
 '대외': 10581,
 '##iam': 7521,
 '나이': 3423,
 '서포터': 24683,
 '열두': 24444,
 '임의': 22292,
 '1주일': 28312,
 '제국을': 19503,
 '분쟁이': 29244,
 '##비로': 9949,
 '##사노': 29763,
 '##2가': 13615,
 '한민': 29104,
 '제13대': 14108,
 '강에': 20695,
 '5화': 27410,
 '플라이': 12115,
 '성적은': 23608,
 '##지역의': 15420,
 '##편으로': 14943,
 '브리트': 29474,
 '제휴': 11456,
 'Family': 25119,
 '위험한': 10646,
 '제작되었다': 15137,
 'Two': 21032,
 '##좌의': 26231,
 '음반을': 16448,
 '번개': 25177,
 '##드볼': 18443,
 '##파이': 8014,
 '복소수': 19810,
 '삼중': 24924,
 '한달': 24966,
 'Bab': 12042,
 '우수학교': 18776,
 '815': 29381,
 '##eh': 27103,
 'Christ': 6931,
 '##캐스터': 28375,
 '##되나': 21040,
 '이노우에': 13028,
 '무슬림': 11746,
 '##대학교': 2107,
 '대광': 25931,
 '마에': 9274,
 '성종': 10728,
 '역임한': 15641,
 '##2의': 13067,
 '지방에서': 16391,
 '자사': 24885,
 '올라가': 11362,
 '레오': 5850,
 '##여자대학교': 8368,
 '길이': 5030,
 '##여고': 14803,
 '출시한': 23397,
 '무대': 7572,
 '##ph': 4452,
 '전쟁에': 9531,
 '양심': 27877,
 '섬으로': 19189,
 '곳곳': 13518,
 '##70': 6151,
 '한국을': 20637,
 '##농민

In [18]:
text = "안녕하세요. 버트를 사용한 모델입니다."

# 토크나이즈한 후 토큰의 id를 출력하라 
token_ids = tokenizer.encode(text).ids
print(token_ids)

# 토크나이즈한 후 각 토큰(string)을 출력하라.
tokens = tokenizer.encode(text).tokens
print(tokens)

[2, 7929, 20936, 16, 509, 3435, 5631, 2842, 5821, 16, 3]
['[CLS]', '안녕', '##하세요', '.', '버', '##트를', '사용한', '모델', '##입니다', '.', '[SEP]']


In [19]:
# 신조어를 토크나이징할 수 있는지 테스트해보자.
text = "킹받네"
tokens = tokenizer.encode(text).tokens
print(tokens)

['[CLS]', '킹', '##받', '##네', '[SEP]']


In [20]:
# 사전에 없는 단어는 어떻게 토크나이즈 되는가?
text = "어쩔티비"
unknown_token_ids = tokenizer.encode(text).ids # 토큰 id
unknown_tokens = tokenizer.encode(text).tokens # 토큰
print(unknown_token_ids)
print(unknown_tokens)

[2, 1, 3]
['[CLS]', '[UNK]', '[SEP]']


### Decoding
- 토큰 id를 원래 문장으로 디코딩하자.

In [21]:
# 원래 문장: "안녕하세요. 버트를 사용한 모델입니다."
tokenizer.decode(token_ids)

'안녕하세요. 버트를 사용한 모델입니다.'

In [22]:
# 사전에 없는 단어는 어떻게 디코딩되는가?
tokenizer.decode(unknown_token_ids)

''